In [1]:
import pandas as pd
import numpy as np
from helpers import *
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import MaxNLocator
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap
from IPython.display import display
from geopy.distance import geodesic
import math
from statsmodels.stats import diagnostic
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
%load_ext autoreload
%autoreload 2

In [2]:
beers = pd.read_csv("dataset/FilteredData/beers.csv")
breweries = pd.read_csv("dataset/FilteredData/breweries.csv")
ratings = pd.read_csv("dataset/FilteredData/ratings.csv", dtype = {'user_id': str})
users = pd.read_csv("dataset/FilteredData/users.csv", dtype = {'user_id': str})

STYLE MOST CONSUMED BY YEAR

In [3]:
all_years_list = sorted(ratings['year'].unique())
style_popularities_by_year = pd.DataFrame()
for year in all_years_list:
    location_style = location_style_stats(ratings.copy(), users.copy(), year, loc_style_threshold = 10, rating_flag = False)
    location_style_groupedby_loc = location_style.groupby('location')
    idx_most_popular = location_style_groupedby_loc['popularity_percentage'].idxmax()
    most_popular_styles_by_location = location_style.loc[idx_most_popular]
    most_popular_styles_by_location = most_popular_styles_by_location[['location', 'style']]
    most_popular_styles_by_location.set_index('location', inplace = True)
    most_popular_styles_by_location = most_popular_styles_by_location.rename(columns = {'style': str(year)})
    style_popularities_by_year = pd.concat([style_popularities_by_year, most_popular_styles_by_location], axis = 1)

style_popularities_by_year

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
location,,,,,,,,,,,,
Australia,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,India Pale Ale (IPA),India Pale Ale (IPA),American Pale Ale,American Pale Ale,India Pale Ale (IPA)
Belgium,Belgian Strong Ale,Belgian Strong Ale,Belgian Strong Ale,Belgian Strong Ale,Belgian Strong Ale,Belgian Strong Ale,Belgian Strong Ale,Belgian Strong Ale,Belgian Ale,Belgian Ale,Belgian Ale,Imperial Stout
Brazil,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager
Canada,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,India Pale Ale (IPA),Pale Lager,American Pale Ale,Saison,Saison,Sour/Wild Ale
Croatia,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pilsener,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager,Pale Lager
...,...,...,...,...,...,...,...,...,...,...,...,...
"United States, Virginia",India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA)
"United States, Washington",India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA)
"United States, West Virginia",India Pale Ale (IPA),Pale Lager,India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),India Pale Ale (IPA),Pale Lager,Saison


STYLE HIGHEST RATED BY YEAR

In [4]:
all_years_list = sorted(ratings['year'].unique())
style_ratings_by_year = pd.DataFrame()
for year in all_years_list:
    location_style = location_style_stats(ratings.copy(), users.copy(), year, loc_style_threshold = 10, rating_flag = True)
    location_style_groupedby_loc = location_style.groupby('location')
    idx_highest_rated = location_style_groupedby_loc['z_score'].idxmax()
    highest_rated_styles_by_location = location_style.loc[idx_highest_rated]
    highest_rated_styles_by_location = highest_rated_styles_by_location[['location', 'style']]
    highest_rated_styles_by_location.set_index('location', inplace = True)
    highest_rated_styles_by_location = highest_rated_styles_by_location.rename(columns = {'style': str(year)})
    style_ratings_by_year = pd.concat([style_ratings_by_year, highest_rated_styles_by_location], axis = 1)

style_ratings_by_year

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
location,,,,,,,,,,,,
Australia,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Eisbock,Abt/Quadrupel,Lambic Style - Gueuze,Imperial Stout,Abt/Quadrupel
Belgium,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Lambic Style - Gueuze,Abt/Quadrupel,Imperial Stout,American Strong Ale,Imperial Stout,Imperial Stout,Imperial Stout
Brazil,German Hefeweizen,Abt/Quadrupel,Abt/Quadrupel,Belgian Strong Ale,Imperial Stout,Lambic Style - Fruit,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abbey Tripel
Canada,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Lambic Style - Gueuze,Abt/Quadrupel,Abt/Quadrupel,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze
Croatia,Belgian Strong Ale,Porter,Abt/Quadrupel,Imperial Stout,Imperial Stout,Imperial Stout,Imperial Stout,Imperial Stout,Imperial Stout,Imperial Stout,Abt/Quadrupel,Lambic Style - Fruit
...,...,...,...,...,...,...,...,...,...,...,...,...
"United States, Virginia",Abt/Quadrupel,Abt/Quadrupel,Sour Red/Brown,Lambic Style - Gueuze,Sour/Wild Ale,Lambic Style - Gueuze,Lambic Style - Unblended,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze
"United States, Washington",Abt/Quadrupel,Abt/Quadrupel,Lambic Style - Gueuze,Sour Red/Brown,Lambic Style - Unblended,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze,Lambic Style - Gueuze
"United States, West Virginia",Doppelbock,Imperial Stout,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Abt/Quadrupel,Imperial Stout,Lambic Style - Fruit,Imperial Stout,Abt/Quadrupel


BREWERY LOCATION MOST CONSUMED BY YEAR

In [5]:
all_years_list = sorted(ratings['year'].unique())
bc_popularities_by_year = pd.DataFrame()
for year in all_years_list:
    location_bc = location_brewery_country_stats(ratings.copy(), users.copy(), breweries.copy(), year, loc_brewery_threshold = 20, rating_flag = False)
    location_bc_gb_loc = location_bc.groupby('location')
    idx_most_popular = location_bc_gb_loc['popularity_percentage'].idxmax()
    most_popular_bc_by_location = location_bc.loc[idx_most_popular]
    most_popular_bc_by_location = most_popular_bc_by_location[['location', 'brewery_location']]
    most_popular_bc_by_location.set_index('location', inplace = True)
    most_popular_bc_by_location = most_popular_bc_by_location.rename(columns = {'brewery_location': str(year)})
    bc_popularities_by_year = pd.concat([bc_popularities_by_year, most_popular_bc_by_location], axis = 1)
    
bc_popularities_by_year

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
location,,,,,,,,,,,,
Australia,Australia,Australia,Australia,Australia,Australia,Australia,Australia,Australia,Australia,Australia,Australia,Australia
Belgium,Belgium,Belgium,Belgium,Belgium,Belgium,Belgium,Belgium,Belgium,Belgium,Belgium,Belgium,Belgium
Brazil,Brazil,Belgium,Brazil,Brazil,Brazil,Brazil,Brazil,Brazil,Brazil,Brazil,Brazil,Brazil
Canada,Canada,Canada,Canada,Canada,Canada,Canada,Canada,Canada,Canada,Canada,Canada,Canada
Croatia,Belgium,Belgium,Germany,Czech Republic,Germany,Germany,Germany,Germany,Germany,Germany,Croatia,Croatia
...,...,...,...,...,...,...,...,...,...,...,...,...
"United States, Virginia","United States, California","United States, California","United States, California","United States, California","United States, California","United States, California","United States, California","United States, California","United States, Virginia","United States, Virginia","United States, Virginia","United States, Virginia"
"United States, Washington","United States, California","United States, California","United States, California","United States, California","United States, California","United States, California","United States, California","United States, California","United States, California","United States, Washington","United States, Washington","United States, Washington"
"United States, West Virginia","United States, Pennsylvania","United States, Pennsylvania","United States, California","United States, New York","United States, California","United States, California","United States, California","United States, California","United States, California","United States, California","United States, California","United States, Ohio"


BREWERY LOCATION HIGHEST RATED BY YEAR

In [6]:
all_years_list = sorted(ratings['year'].unique())
bc_ratings_by_year = pd.DataFrame()
for year in all_years_list:
    location_bc = location_brewery_country_stats(ratings.copy(), users.copy(), breweries.copy(), year, loc_brewery_threshold = 20, rating_flag = True)
    location_bc_gb_loc = location_bc.groupby('location')
    idx_highest_rated = location_bc_gb_loc['z_score'].idxmax()
    highest_rated_bc_by_location = location_bc.loc[idx_highest_rated]
    highest_rated_bc_by_location = highest_rated_bc_by_location[['location', 'brewery_location']]
    highest_rated_bc_by_location.set_index('location', inplace=True)
    highest_rated_bc_by_location = highest_rated_bc_by_location.rename(columns={'brewery_location': str(year)})
    bc_ratings_by_year = pd.concat([bc_ratings_by_year, highest_rated_bc_by_location], axis=1)
    
bc_ratings_by_year

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
location,,,,,,,,,,,,
Australia,"United States, California","United States, Colorado","United States, Oregon",Norway,Norway,"United States, Michigan","United States, Alaska",Norway,"United States, Vermont",Norway,"United States, Oklahoma","United States, Massachusetts"
Belgium,"United States, California","United States, California","United States, Michigan","United States, Georgia","United States, Oregon",Norway,"United States, Illinois","United States, Vermont","United States, Iowa","United States, Vermont","United States, South Carolina","United States, Ohio"
Brazil,Germany,Canada,"United States, California",Belgium,Belgium,"United States, California",Belgium,"United States, Indiana","United States, Florida","United States, Illinois","United States, Florida","United States, Massachusetts"
Canada,"United States, Alaska",Norway,Norway,Norway,Norway,Norway,"United States, Indiana","United States, Iowa","United States, Vermont","United States, Oklahoma","United States, Iowa","United States, Oklahoma"
Croatia,Belgium,Belgium,Belgium,England,Scotland,Sweden,Belgium,Scotland,Norway,Norway,Norway,"United States, Oregon"
...,...,...,...,...,...,...,...,...,...,...,...,...
"United States, Virginia","United States, Connecticut","United States, Connecticut",Norway,Norway,"United States, Minnesota","United States, Alaska","United States, Alaska","United States, Iowa","United States, Iowa",Norway,Switzerland,"United States, Oklahoma"
"United States, Washington","United States, Connecticut",Norway,Norway,Norway,"United States, Indiana",Switzerland,Belgium,"United States, Iowa","United States, Iowa","United States, Vermont","United States, Iowa","United States, Wyoming"
"United States, West Virginia","United States, Ohio",Belgium,"United States, Michigan",Belgium,Belgium,Belgium,"United States, Michigan",Belgium,Belgium,"United States, Indiana",Belgium,"United States, Oregon"


PREFERENCES FOR THE MAP (WITH CLUSTERING)

In [7]:
beer_styles_dict = {'Wheat Beer': ['German Hefeweizen', 'Dunkelweizen', 'Witbier', 'Berliner Weisse', 'Grodziskie/Gose/Lichtenhainer', 'German Kristallweizen', 'Wheat Ale'], 'Lambic & Sour Ale': ['Sour/Wild Ale', 'Fruit Beer', 'Lambic Style - Fruit', 'Lambic Style - Gueuze', 'Lambic Style - Unblended', 'Lambic Style - Faro', 'Sour Red/Brown'], 'Belgian Ale': ['Abbey Tripel', 'Belgian Ale', 'Belgian Strong Ale', 'Abbey Dubbel', 'Abt/Quadrupel', 'Bière de Garde', 'Saison'], 'Pale Ale': ['Golden Ale/Blond Ale', 'American Pale Ale', 'India Pale Ale (IPA)', 'Black IPA', 'Amber Ale', 'Imperial IPA', 'Irish Ale'], 'Bitter': ['Bitter', 'Premium Bitter/ESB'], 'Scottish Ale': ['Scotch Ale', 'Scottish Ale'], 'Brown Ale': ['Mild Ale', 'Brown Ale', 'American Strong Ale ', 'English Strong Ale', 'Old Ale'], 'Porter': ['Porter', 'Baltic Porter'], 'Stout': ['Stout', 'Sweet Stout', 'Dry Stout', 'Foreign Stout', 'Imperial Stout'], 'Pilsener': ['Pilsener', 'Dortmunder/Helles'], 'Lager': ['Imperial Pils/Strong Pale Lager', 'Pale Lager', 'Amber Lager/Vienna', 'Schwarzbier', 'Altbier', 'Zwickel/Keller/Landbier', 'Dunkel/Tmavý'], 'Bock': ['Doppelbock', 'Dunkler Bock', 'Eisbock', 'Heller Bock', 'Weizen Bock'], 'American Special': ['Cream Ale', 'California Common'], 'Alternative/Traditional': ['Sahti/Gotlandsdricke/Koduõlu', 'Mead', 'Malt Liquor', 'Low Alcohol', 'Specialty Grain', 'Spice/Herb/Vegetable', 'Oktoberfest/Märzen', 'Smoked', 'Barley Wine', 'Kölsch', 'Traditional Ale']}

inverted_beer_styles_dict = {}

for key, values in beer_styles_dict.items():
    for value in values:
        inverted_beer_styles_dict[value] = key

location_list_1 = pd.unique(bc_popularities_by_year.values.flatten())
location_list_2 = pd.unique(bc_ratings_by_year.values.flatten())
location_set = set(location_list_1.tolist() + location_list_2.tolist())
us_locations = [location for location in list(location_set) if 'United States' in location]
location_dict = {'Northern Europe': ['Denmark', 'Estonia', 'Finland', 'Iceland', 'Norway', 'Sweden'], 'Central/Western Europe': ['France', 'Germany', 'Ireland', 'Netherlands', 'Belgium', 'Switzerland', 'Czech Republic'], 'Southern Europe': ['Italy', 'Spain', 'Croatia'], 'Eastern Europe': ['Poland', 'Russia'], 'United Kingdom': ['England', 'Scotland', 'Northern Ireland'], 'North America': us_locations + ['Canada'], 'Central and South America': ['Brazil', 'Trinidad & Tobago'], 'Oceania': ['Australia', 'New Zealand']}

inverted_location_dict = {}

for key, values in location_dict.items():
    for value in values:
        inverted_location_dict[value] = key

STYLE POPULARITIES - CLUSTERED

In [8]:
style_popularities_by_year_clustered = style_popularities_by_year.replace(inverted_beer_styles_dict)
style_popularities_by_year_clustered

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
location,,,,,,,,,,,,
Australia,Lager,Lager,Lager,Lager,Lager,Lager,Lager,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale
Belgium,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Stout
Brazil,Lager,Lager,Lager,Lager,Lager,Lager,Lager,Lager,Lager,Lager,Lager,Lager
Canada,Lager,Lager,Lager,Lager,Lager,Lager,Pale Ale,Lager,Pale Ale,Belgian Ale,Belgian Ale,Lambic & Sour Ale
Croatia,Lager,Lager,Lager,Lager,Pilsener,Lager,Lager,Lager,Lager,Lager,Lager,Lager
...,...,...,...,...,...,...,...,...,...,...,...,...
"United States, Virginia",Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale
"United States, Washington",Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale
"United States, West Virginia",Pale Ale,Lager,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Pale Ale,Lager,Belgian Ale


STYLE HIGHEST RATED - CLUSTERED

In [9]:
style_ratings_by_year_clustered = style_ratings_by_year.replace(inverted_beer_styles_dict)
style_ratings_by_year_clustered

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
location,,,,,,,,,,,,
Australia,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Bock,Belgian Ale,Lambic & Sour Ale,Stout,Belgian Ale
Belgium,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Lambic & Sour Ale,Belgian Ale,Stout,Brown Ale,Stout,Stout,Stout
Brazil,Wheat Beer,Belgian Ale,Belgian Ale,Belgian Ale,Stout,Lambic & Sour Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale
Canada,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Lambic & Sour Ale,Belgian Ale,Belgian Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale
Croatia,Belgian Ale,Porter,Belgian Ale,Stout,Stout,Stout,Stout,Stout,Stout,Stout,Belgian Ale,Lambic & Sour Ale
...,...,...,...,...,...,...,...,...,...,...,...,...
"United States, Virginia",Belgian Ale,Belgian Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale
"United States, Washington",Belgian Ale,Belgian Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale,Lambic & Sour Ale
"United States, West Virginia",Bock,Stout,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Belgian Ale,Stout,Lambic & Sour Ale,Stout,Belgian Ale


BREWERY LOCATION POPULARITIES - CLUSTERED

In [10]:
bc_popularities_by_year_clustered = bc_popularities_by_year.replace(inverted_location_dict)
bc_popularities_by_year_clustered

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
location,,,,,,,,,,,,
Australia,Oceania,Oceania,Oceania,Oceania,Oceania,Oceania,Oceania,Oceania,Oceania,Oceania,Oceania,Oceania
Belgium,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe
Brazil,Central and South America,Central/Western Europe,Central and South America,Central and South America,Central and South America,Central and South America,Central and South America,Central and South America,Central and South America,Central and South America,Central and South America,Central and South America
Canada,North America,North America,North America,North America,North America,North America,North America,North America,North America,North America,North America,North America
Croatia,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Central/Western Europe,Southern Europe,Southern Europe
...,...,...,...,...,...,...,...,...,...,...,...,...
"United States, Virginia",North America,North America,North America,North America,North America,North America,North America,North America,North America,North America,North America,North America
"United States, Washington",North America,North America,North America,North America,North America,North America,North America,North America,North America,North America,North America,North America
"United States, West Virginia",North America,North America,North America,North America,North America,North America,North America,North America,North America,North America,North America,North America


BREWERY LOCATION HIGHEST RATED - CLUSTERED

In [11]:
bc_ratings_by_year_clustered = bc_ratings_by_year.replace(inverted_location_dict)
bc_ratings_by_year_clustered

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
location,,,,,,,,,,,,
Australia,North America,North America,North America,Northern Europe,Northern Europe,North America,North America,Northern Europe,North America,Northern Europe,North America,North America
Belgium,North America,North America,North America,North America,North America,Northern Europe,North America,North America,North America,North America,North America,North America
Brazil,Central/Western Europe,North America,North America,Central/Western Europe,Central/Western Europe,North America,Central/Western Europe,North America,North America,North America,North America,North America
Canada,North America,Northern Europe,Northern Europe,Northern Europe,Northern Europe,Northern Europe,North America,North America,North America,North America,North America,North America
Croatia,Central/Western Europe,Central/Western Europe,Central/Western Europe,United Kingdom,United Kingdom,Northern Europe,Central/Western Europe,United Kingdom,Northern Europe,Northern Europe,Northern Europe,North America
...,...,...,...,...,...,...,...,...,...,...,...,...
"United States, Virginia",North America,North America,Northern Europe,Northern Europe,North America,North America,North America,North America,North America,Northern Europe,Central/Western Europe,North America
"United States, Washington",North America,Northern Europe,Northern Europe,Northern Europe,North America,Central/Western Europe,Central/Western Europe,North America,North America,North America,North America,North America
"United States, West Virginia",North America,Central/Western Europe,North America,Central/Western Europe,Central/Western Europe,Central/Western Europe,North America,Central/Western Europe,Central/Western Europe,North America,Central/Western Europe,North America
